In [1]:
import os
from pathlib import Path
import datetime as dt

import json
import xarray as xr
import numpy as np
import pandas as pd
from netCDF4 import Dataset as netcdf_dataset

import cmocean
import matplotlib as mpl

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from matplotlib.transforms import offset_copy

from math import floor
import ipywidgets as widgets

In [2]:
class Settings(object):
    _instance = None
    _fname = "test_settings.json"
    _data = {}

    def set_config(self,_fname):
        with open(_fname) as json_file:
            self._data = json.load(json_file)
        if not os.path.exists(self._data['data_dir']):
            os.makedirs(self._data['data_dir'])
        if not os.path.exists(self._data['out_dir']):
            os.makedirs(self._data['out_dir'])

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(Settings, cls).__new__(cls)
            cls.set_config(cls, cls._fname)
        return cls._instance
    def get(self,atr):
        return self._data[atr]
    def set_attribute(self,atr, value):
        self._data[atr] = value

In [3]:
#file list is []
def get_min_max(wave_dict, file_list):
    wave_min, wave_max = 0, 0
    for f_p in file_list:
        dataset = netcdf_dataset(os.path.join(Settings().get('data_dir'), f_p))
        for wave in wave_dict.keys():
            wave_data = dataset.variables[wave][0, :, :]
            #wtf why i have temp here?
            temp_min = wave_data.min()
            temp_max = wave_data.max()
            if temp_min < wave_min:
                wave_min = temp_min
            if temp_max > wave_max:
                wave_max = temp_max
        dataset.close()
    return (wave_min, wave_max)


In [4]:
def make_ticks(w_min, w_max):
    scale = w_max - w_min
    if scale > 10:
        num = floor((w_max - w_min)/2) + 1
    else:
        num = floor(w_max) + 1
    ticks = np.linspace(floor(w_min), floor(w_max), num=num, dtype = int)
    labels = []
    for t in ticks:
        if t == 0:
            labels.append('flat')
        elif t == 1:
            labels.append('1 meter')
        else:
            labels.append(str(t)+ ' meters')
    return ticks, labels

In [5]:
#just convert index to actual datime string eg:  '2021.01.04 00h:00'
def get_time_string(i_time):
    return pd.to_datetime(str(time_xr[i_time].data)).strftime('%Y.%m.%d %Hh:%M')

In [6]:
wave_dict = {"VHM0_WW": "significant wind wave height",
            "VHM0_SW1": "significant primary swell wave height",
            "VHM0_SW2": "significant secondary swell wave height"}

#load data
stgs = Settings()
data_file = 'testfile1.nc'
file_path = os.path.join(stgs.get('data_dir'),data_file)
data_list = [file_path] 
dataset = netcdf_dataset(file_path)
dataset_xr = xr.open_dataset(file_path)

lats = dataset.variables['latitude'][:]
lons = dataset.variables['longitude'][:]
time_xr = dataset_xr['time']
proj = ccrs.PlateCarree()

(w_min, w_max) = get_min_max(wave_dict, data_list)

cmap = cmocean.cm.deep
levels = MaxNLocator(nbins=10).tick_values(w_min, w_max)
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
ticks,labels = make_ticks(w_min, w_max)
dx, dy = 0.05, 0.05

In [7]:
def jupyter_single_frame_ultra1(i_time):
    wave_var = "VHM0_WW"
    wave_name = wave_dict[wave_var]
    wave_tn = dataset.variables[wave_var][0, :, :]
        
    wave_tn = dataset.variables[wave_var][i_time, :, :]
    fig = plt.figure(figsize=(12, 6), clear=False)
    
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    
    ax.set_extent([stgs.get('min_lon'),stgs.get('max_lon'),stgs.get('min_lat'),stgs.get('max_lat')], crs=proj)
    geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
    text_transform = offset_copy(geodetic_transform, units='dots', x=-25)

    ax.plot(stgs.get('point_lon'), stgs.get('point_lat'), marker='o', color='red', markersize=8,
            alpha=0.7, transform=ccrs.Geodetic())

    ax.text(stgs.get('text_lon'), stgs.get('text_lat'), stgs.get('text_label'),
            verticalalignment='bottom', horizontalalignment='left',
            transform=text_transform,
            bbox=dict(facecolor='white', alpha=0.5, boxstyle='round'))

    ax.set_title(wave_name + " " + get_time_string(i_time), fontsize=16)
    
    gr= ax.gridlines(draw_labels = True)
    gr.right_labels = False
    ax.coastlines()
    ax.plot(stgs.get('point_lon'), stgs.get('point_lat'), marker='o', color='red', markersize=8,
            alpha=0.7, transform=ccrs.Geodetic(), animated=True)
    cs = plt.contour(lons, lats, wave_tn, levels = levels, colors='w', transform=ccrs.PlateCarree())
    ax.clabel(cs, colors='w', fontsize=20)
    contour_img = ax.contourf(lons + dx/2., lats + dy/2.,wave_tn, levels=levels, cmap=cmap)
    cbar = fig.colorbar(contour_img, ax=ax, ticks = ticks)
    cbar.ax.set_yticklabels(labels)
    
    plt.show(block=False)
  
def jupyter_single_frame_ultra2(i_time):
    wave_var = "VHM0_SW1"
    wave_name = wave_dict[wave_var]
    wave_tn = dataset.variables[wave_var][0, :, :]

    
    wave_tn = dataset.variables[wave_var][i_time, :, :]
    fig = plt.figure(figsize=(12, 6), clear=False)
    
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    
    ax.set_extent([stgs.get('min_lon'),stgs.get('max_lon'),stgs.get('min_lat'),stgs.get('max_lat')], crs=proj)
    geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
    text_transform = offset_copy(geodetic_transform, units='dots', x=-25)

    ax.plot(stgs.get('point_lon'), stgs.get('point_lat'), marker='o', color='red', markersize=8,
            alpha=0.7, transform=ccrs.Geodetic())

    ax.text(stgs.get('text_lon'), stgs.get('text_lat'), stgs.get('text_label'),
            verticalalignment='bottom', horizontalalignment='left',
            transform=text_transform,
            bbox=dict(facecolor='white', alpha=0.5, boxstyle='round'))

    ax.set_title(wave_name + " " + get_time_string(i_time), fontsize=16)
    
    gr= ax.gridlines(draw_labels = True)
    gr.right_labels = False
    ax.coastlines()
    ax.plot(stgs.get('point_lon'), stgs.get('point_lat'), marker='o', color='red', markersize=8,
            alpha=0.7, transform=ccrs.Geodetic(), animated=True)
    cs = plt.contour(lons, lats, wave_tn, levels = levels, colors='w', transform=ccrs.PlateCarree())
    ax.clabel(cs, colors='w', fontsize=20)
    contour_img = ax.contourf(lons + dx/2., lats + dy/2.,wave_tn, levels=levels, cmap=cmap)
    cbar = fig.colorbar(contour_img, ax=ax, ticks = ticks)
    cbar.ax.set_yticklabels(labels)
    
    plt.show(block=False)

def jupyter_single_frame_ultra3(i_time):
    wave_var = "VHM0_SW2"
    wave_name = wave_dict[wave_var]
    wave_tn = dataset.variables[wave_var][0, :, :]
    
    wave_tn = dataset.variables[wave_var][i_time, :, :]
    fig = plt.figure(figsize=(12, 6), clear=False)
    
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    
    ax.set_extent([stgs.get('min_lon'),stgs.get('max_lon'),stgs.get('min_lat'),stgs.get('max_lat')], crs=proj)
    geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
    text_transform = offset_copy(geodetic_transform, units='dots', x=-25)

    ax.plot(stgs.get('point_lon'), stgs.get('point_lat'), marker='o', color='red', markersize=8,
            alpha=0.7, transform=ccrs.Geodetic())

    ax.text(stgs.get('text_lon'), stgs.get('text_lat'), stgs.get('text_label'),
            verticalalignment='bottom', horizontalalignment='left',
            transform=text_transform,
            bbox=dict(facecolor='white', alpha=0.5, boxstyle='round'))

    ax.set_title(wave_name + " " + get_time_string(i_time), fontsize=16)
    
    gr= ax.gridlines(draw_labels = True)
    gr.right_labels = False
    ax.coastlines()
    ax.plot(stgs.get('point_lon'), stgs.get('point_lat'), marker='o', color='red', markersize=8,
            alpha=0.7, transform=ccrs.Geodetic(), animated=True)
    cs = plt.contour(lons, lats, wave_tn, levels = levels, colors='w', transform=ccrs.PlateCarree())
    ax.clabel(cs, colors='w', fontsize=20)
    contour_img = ax.contourf(lons + dx/2., lats + dy/2.,wave_tn, levels=levels, cmap=cmap)
    cbar = fig.colorbar(contour_img, ax=ax, ticks = ticks)
    cbar.ax.set_yticklabels(labels)
    
    plt.show(block=False)
   

In [8]:




interactive_plot = widgets.interactive(jupyter_single_frame_ultra1,  i_time = widgets.IntSlider(min=0, max=len(time_xr.time)-1, step=1, value=0))
output = interactive_plot.children[-1]
output.layout.height = '400px'
output.layout.weight = '300px'
interactive_plot

interactive(children=(IntSlider(value=0, description='i_time', max=12), Output(layout=Layout(height='400px')))…

In [9]:
wave_var = "VHM0_SW1"
wave_name = wave_dict[wave_var]


interactive_plot = widgets.interactive(jupyter_single_frame_ultra2,  i_time = widgets.IntSlider(min=0, max=len(time_xr.time)-1, step=1, value=0))
output = interactive_plot.children[-1]
output.layout.height = '400px'
output.layout.weight = '300px'
interactive_plot

interactive(children=(IntSlider(value=0, description='i_time', max=12), Output(layout=Layout(height='400px')))…

In [10]:
wave_var = "VHM0_SW2"
wave_name = wave_dict[wave_var]


interactive_plot = widgets.interactive(jupyter_single_frame_ultra3,  i_time = widgets.IntSlider(min=0, max=len(time_xr.time)-1, step=1, value=0))
output = interactive_plot.children[-1]
output.layout.height = '400px'
output.layout.weight = '300px'
interactive_plot

interactive(children=(IntSlider(value=0, description='i_time', max=12), Output(layout=Layout(height='400px')))…

In [11]:
dataset

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.6
    time_coverage_start: 20210111-03:00:00
    time_coverage_end: 20210112-00:00:00
    date_created: 20210102-08:32:00
    product_type: forecast
    product: GLOBAL_ANALYSIS_FORECAST_WAV_001_027
    product_ref_date: 20210102-00:00:00
    product_range: D+9
    product_user_manual: http://marine.copernicus.eu/documents/PUM/CMEMS-GLO-PUM-001-027.pdf
    quality_information_document:  http://marine.copernicus.eu/documents/QUID/CMEMS-GLO-QUID-001-027.
    dataset: global-analysis-forecast-wav-001-027
    title: Mean fields from global wave model MFWAM of Meteo-France with ECMWF forcing
    institution: METEO-FRANCE
    references: http://marine.copernicus.eu
    credit: E.U. Copernicus Marine Service Information (CMEMS)
    licence: http://marine.copernicus.eu/services-portfolio/service-commitments-and
    contact: servicedesk.cmems@mercator-ocean.eu
    producer: CME